In [3]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 11.6 MB/s 
     |████████████████████████████████| 181 kB 53.4 MB/s 
     |████████████████████████████████| 145 kB 58.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a2cb93e175217b6dad482e3f9343bbb5b625ff08dafcfa0425f50095e96d4f8d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [4]:
import os
import numpy as np
import torch
from torchvision.datasets.mnist import MNIST
from torch.utils.data.dataset import Dataset
from PIL import Image, ImageDraw, ImageOps
import json
import math
import logging
import torch
from torch.nn import functional as F
import torch.nn as nn
import random
import seaborn as sns
import wandb
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset

In [5]:
logger = logging.getLogger(__name__)

In [6]:
!nvidia-smi

Wed May 11 18:35:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def gen_colors(num_colors):
    palette = sns.color_palette(None, num_colors)
    rgb_triples = [[int(x[0]*255), int(x[1]*255), int(x[2]*255)] for x in palette]
    return rgb_triples

def trim_tokens(tokens, bos, eos, pad=None):
    bos_idx = np.where(tokens == bos)[0]
    tokens = tokens[bos_idx[0]+1:] if len(bos_idx) > 0 else tokens
    eos_idx = np.where(tokens == eos)[0]
    tokens = tokens[:eos_idx[0]] if len(eos_idx) > 0 else tokens
    # tokens = tokens[tokens != bos]
    # tokens = tokens[tokens != eos]
    if pad is not None:
        tokens = tokens[tokens != pad]
    return tokens

In [8]:
class Padding(object):
    def __init__(self, max_length, vocab_size):
        self.max_length = max_length
        self.bos_token = vocab_size - 3
        self.eos_token = vocab_size - 2
        self.pad_token = vocab_size - 1

    def __call__(self, layout):
        # grab a chunk of (max_length + 1) from the layout

        chunk = torch.zeros(self.max_length+1, dtype=torch.long) + self.pad_token
        # Assume len(item) will always be <= self.max_length:
        chunk[0] = self.bos_token
        chunk[1:len(layout)+1] = layout
        chunk[len(layout)+1] = self.eos_token

        x = chunk[:-1]
        y = chunk[1:]
        return {'x': x, 'y': y}

In [9]:
def compute_overlap_coverage(boxes):
    area = [(item[2] - item[0]) * (item[3] - item[1]) for item in boxes]
    overlaps = 0
    good_areas = [(item[2] - item[0]) * (item[3] - item[1]) for item in boxes]
    boxes = sorted(boxes, key=lambda x: x[0])
    for i in range(len(boxes) - 1):
        for j in range(i + 1, len(boxes)):
            if i == j:
                continue
            if boxes[j][0] >= boxes[i][2] or boxes[j][1] >= boxes[i][3] or boxes[i][1] >= boxes[j][3]:
                continue
            #print("boxes[i] is ", boxes[i])
            #print("boxes[j] is ", boxes[j])
            end0 = max(boxes[i][0], boxes[j][0])
            #print("end0 is ", end0)
            end1 = max(boxes[i][1], boxes[j][1])
            #print("end1 is ", end1)
            end2 = min(boxes[i][2], boxes[j][2])
            #print("end2 is ", end2)
            end3 = min(boxes[i][3], boxes[j][3])
            #print("end3 is ", end3)
            overlap = (end2 - end0) * (end3 - end1)
            overlaps += overlap
            good_areas[i] -= overlap
            good_areas[j] -= overlap
    
    union = sum(good_areas) + overlaps
    overlap_rate = min(1, overlaps / union)
    '''
    if overlap_rate < 0 or overlap_rate > 1:
        print("error_case found")
        print("overlaps is ", overlaps)
        print("sum(good_areas) is ", sum(good_areas))
        print("union is ", union)
        print("good_areas is ", good_areas)
        print("boxes is ", boxes)
        print(nbnn)
    '''
    coverage = union / (256 * 256)
    return overlap_rate, coverage
    


In [10]:
class JSONLayout(Dataset):
    def __init__(self, json_path, max_length=None, precision=8):
        with open(json_path, "r") as f:
            data = json.loads(f.read())
        images, annotations, categories = data['images'], data['annotations'], data['categories']
        convert_dict = {1: 2, 2: 1, 3: 5, 4: 1, 5: 1, 6: 5, 7: 4}
        categories = [{'id': 1, 'name': 'text', 'supercategory': ''},
                      {'id': 2, 'name': 'title', 'supercategory': ''},
                      {'id': 3, 'name': 'list', 'supercategory': ''},
                      {'id': 4, 'name': 'table', 'supercategory': ''},
                      {'id': 5, 'name': 'figure', 'supercategory': ''}]
        for item in annotations:
            item['category_id'] = convert_dict[item['category_id']]
        self.size = pow(2, precision)
        self.categories = {c["id"]: c for c in categories}
        self.colors = gen_colors(len(self.categories))
        self.json_category_id_to_contiguous_id = {
            v: i + self.size for i, v in enumerate([c["id"] for c in self.categories.values()])
        }
        self.contiguous_category_id_to_json_id = {
            v: k for k, v in self.json_category_id_to_contiguous_id.items()
        }
        self.vocab_size = self.size + len(self.categories) + 3  # bos, eos, pad tokens
        self.bos_token = self.vocab_size - 3
        self.eos_token = self.vocab_size - 2
        self.pad_token = self.vocab_size - 1

        image_to_annotations = {}
        for annotation in annotations:
            image_id = annotation["image_id"]
            if not (image_id in image_to_annotations):
                image_to_annotations[image_id] = []
            image_to_annotations[image_id].append(annotation)
        #anno_values = list(image_to_annotations.values())
        #lengths = np.array([len(item) for item in anno_values])
        #print("mean lengths is ", np.mean(lengths))
        self.data = []
        for image in images:
            image_id = image["id"]
            height, width = float(image["height"]), float(image["width"])
            if image_id not in image_to_annotations:
                continue
            ann_box = []
            ann_cat = []
            for ann in image_to_annotations[image_id]:
                x, y, w, h = ann["bbox"]
                ann_box.append([x, y, w, h])
                ann_cat.append(self.json_category_id_to_contiguous_id[ann["category_id"]])
            # Sort boxes
            ann_box = np.array(ann_box)
            ind = np.lexsort((ann_box[:, 0], ann_box[:, 1]))
            ann_box = ann_box[ind]
            # Discretize boxes
            #print("ann_box is ", ann_box[:20])
            ann_box = self.quantize_box(ann_box, width, height)
            #print("ann_box after quantize is ", ann_box[:20])
            # Append the categories
            ann_cat = np.array(ann_cat)
            #print("ann_cat is ", ann_cat[:20])
            layout = np.concatenate([ann_cat.reshape(-1, 1), ann_box], axis=1)
            #print("layout is ", layout)
            # Flatten and add to the dataset
            self.data.append(layout.reshape(-1))

        #self.max_length = max_length
        self.max_length = 517
        if self.max_length is None:
            self.max_length = max([len(x) for x in self.data]) + 2  # bos, eos tokens
        self.transform = Padding(self.max_length, self.vocab_size)

    def quantize_box(self, boxes, width, height):

        # range of xy is [0, large_side-1]
        # range of wh is [1, large_side]
        # bring xywh to [0, 1]
        boxes = boxes.astype(np.float32)
        #print("width, height is ", width, height)
        #print("boxes[:, [2, 3]] is ", boxes[:, [2, 3]])
        boxes[:, [2, 3]] = boxes[:, [2, 3]] - 1
        #print("boxes[:, [2, 3]] after transform is ", boxes[:, [2, 3]])
        #print("boxes[:, [0, 2]] is ", boxes[:, [0, 2]])
        boxes[:, [0, 2]] = boxes[:, [0, 2]] / (width - 1)
        #print("boxes[:, [0, 2]] after transform is ", boxes[:, [0, 2]])
        #print("boxes[:, [1, 3]] is ", boxes[:, [1, 3]])
        boxes[:, [1, 3]] = boxes[:, [1, 3]] / (height - 1)
        #print("boxes[:, [1, 3]] after transform is ", boxes[:, [1, 3]])
        boxes = np.clip(boxes, 0, 1)

        # next take xywh to [0, size-1]
        boxes = (boxes * (self.size - 1)).round()

        return boxes.astype(np.int32)

    def __len__(self):
        return len(self.data)

    def render(self, layout):
        img = Image.new('RGB', (256, 256), color=(255, 255, 255))
        draw = ImageDraw.Draw(img, 'RGBA')
        layout = layout.reshape(-1)
        layout = trim_tokens(layout, self.bos_token, self.eos_token, self.pad_token)
        layout = layout[: len(layout) // 5 * 5].reshape(-1, 5)
        box = layout[:, 1:].astype(np.float32)
        box[:, [0, 1]] = box[:, [0, 1]] / (self.size - 1) * 255
        box[:, [2, 3]] = box[:, [2, 3]] / self.size * 256
        #box[:, [0, 1]] = box[:, [0, 1]] * 255
        #box[:, [2, 3]] = box[:, [2, 3]] * 256
        box[:, [2, 3]] = box[:, [0, 1]] + box[:, [2, 3]]
        for i in range(len(layout)):
            x1, y1, x2, y2 = box[i]
            cat = layout[i][0]
            col = self.colors[cat-self.size] if 0 <= cat-self.size < len(self.colors) else [0, 0, 0]
            #print("col is ", col)
            draw.rectangle([x1, y1, x2, y2],
                           outline=tuple(col) + (200,),
                           fill=tuple(col) + (64,),
                           width=2)
        #print("box is ", box)
        # Add border around image
        img = ImageOps.expand(img, border=2)
        overlap_rate, coverage = compute_overlap_coverage(box)

        return [img, overlap_rate, coverage]

    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) tokens from the data
        layout = torch.tensor(self.data[idx], dtype=torch.long)
        layout = self.transform(layout)
        return layout['x'], layout['y']

In [11]:
class GPTConfig:
    """ base GPT config, params common to all GPT versions """
    embd_pdrop = 0.1
    resid_pdrop = 0.1
    attn_pdrop = 0.1

    def __init__(self, vocab_size, block_size, **kwargs):
        self.vocab_size = vocab_size
        self.block_size = block_size
        for k,v in kwargs.items():
            setattr(self, k, v)

In [12]:
class CausalSelfAttention(nn.Module):
    """
    A vanilla multi-head masked self-attention layer with a projection at the end.
    It is possible to use torch.nn.MultiheadAttention here but I am including an
    explicit implementation here to show that there is nothing too scary here.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads
        self.key = nn.Linear(config.n_embd, config.n_embd)
        self.query = nn.Linear(config.n_embd, config.n_embd)
        self.value = nn.Linear(config.n_embd, config.n_embd)
        # regularization
        self.attn_drop = nn.Dropout(config.attn_pdrop)
        self.resid_drop = nn.Dropout(config.resid_pdrop)
        # output projection
        self.proj = nn.Linear(config.n_embd, config.n_embd)
        # causal mask to ensure that attention is only applied to the left in the input sequence
        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head

    def forward(self, x, layer_past=None):
        B, T, C = x.size()

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        k = self.key(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = self.query(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = self.value(x).view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_drop(att)
        y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_drop(self.proj(y))
        return y

In [13]:
class Block(nn.Module):
    """ an unassuming Transformer block """

    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.resid_pdrop),
        )

    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

In [14]:
class GPT(nn.Module):
    """  the full GPT language model, with a context size of block_size """

    def __init__(self, config):
        super().__init__()

        # input embedding stem
        self.tok_emb = nn.Embedding(config.vocab_size, config.n_embd)
        self.pos_emb = nn.Parameter(torch.zeros(1, config.block_size, config.n_embd))
        self.drop = nn.Dropout(config.embd_pdrop)
        # transformer
        self.blocks = nn.Sequential(*[Block(config) for _ in range(config.n_layer)])
        # decoder head
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        self.block_size = config.block_size
        self.apply(self._init_weights)

        logger.info("number of parameters: %e", sum(p.numel() for p in self.parameters()))

    def get_block_size(self):
        return self.block_size

    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, nn.Linear) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def configure_optimizers(self, train_config):
        """
        This long function is unfortunately doing something very simple and is being very defensive:
        We are separating out all parameters of the model into two buckets: those that will experience
        weight decay for regularization and those that won't (biases, and layernorm/embedding weights).
        We are then returning the PyTorch optimizer object.
        """

        # separate out all parameters to those that will and won't experience regularizing weight decay
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn # full param name

                if pn.endswith('bias'):
                    # all biases will not be decayed
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    # weights of whitelist modules will be weight decayed
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    # weights of blacklist modules will NOT be weight decayed
                    no_decay.add(fpn)

        # special case the position embedding parameter in the root GPT module as not decayed
        no_decay.add('pos_emb')

        # validate that we considered every parameter
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        assert len(inter_params) == 0, "parameters %s made it into both decay/no_decay sets!" % (str(inter_params), )
        assert len(param_dict.keys() - union_params) == 0, "parameters %s were not separated into either decay/no_decay set!" \
                                                    % (str(param_dict.keys() - union_params), )

        # create the pytorch optimizer object
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": train_config.weight_decay},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},
        ]
        optimizer = torch.optim.AdamW(optim_groups, lr=train_config.learning_rate, betas=train_config.betas)
        return optimizer

    def forward(self, idx, targets=None, pad_token=-100):
        b, t = idx.size()
        assert t <= self.block_size, "Cannot forward, model block size is exhausted."

        # forward the GPT model
        token_embeddings = self.tok_emb(idx) # each index maps to a (learnable) vector
        position_embeddings = self.pos_emb[:, :t, :] # each position maps to a (learnable) vector
        x = self.drop(token_embeddings + position_embeddings)
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        # if we are given some desired targets also calculate the loss
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=pad_token)

        return logits, loss

In [15]:
args = {
    #"train_data": "/content/drive/MyDrive/Directed_Study/DeepLayout-main/Publeynet/train.json",
    "train_data": "/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/train_dataset_nc.json",
    #"val_data": "/content/drive/MyDrive/Directed_Study/DeepLayout-main/Publeynet/val.json",
    "val_data": "/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/val_dataset_nc.json",
    "max_length": 128,
    "precision": 8,
    'element_order': 'raster',
    'attribute_order': 'cxywh',
    "seed": 42,
    "epochs": 100,
    "batch_size": 16,
    "lr": 4.5e-06, 
    'n_layer': 6,
    'n_embd': 512,
    'n_head': 8,
    'warmup_iters': 0,
    'final_iters': 0,
    'sample_every': 1,
    "exp": "layout",
    "log_dir": "/content/finetune_logs",
    "lr_decay": False
}

In [16]:
train_dataset = JSONLayout(args["train_data"])
valid_dataset = JSONLayout(args["val_data"], max_length=train_dataset.max_length)

In [17]:
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.max_length, 
                  n_layer=args["n_layer"], n_head=args["n_head"], n_embd=args["n_embd"])  # a GPT-1
model = GPT(mconf)

In [18]:
model.load_state_dict(torch.load("/content/checkpoint.pth"))

<All keys matched successfully>

In [19]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[:, [-1]]] = -float('Inf')
    return out

In [20]:
@torch.no_grad()
def sample(model, x, steps, temperature=1.0, sample=False, top_k=None):
    """
    take a conditioning sequence of indices in x (of shape (b,t)) and predict the next token in
    the sequence, feeding the predictions back into the model each time. Clearly the sampling
    has quadratic complexity unlike an RNN that is only linear, and has a finite context window
    of block_size, unlike an RNN that has an infinite context window.
    """
    block_size = model.module.get_block_size() if hasattr(model, "module") else model.getcond_block_size()
    model.eval()
    for k in range(steps):
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:]  # crop context if needed
        logits, _ = model(x_cond)
        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)
        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)

    return x

In [21]:
class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1  # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_iters = 0
    final_iters = 0  # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_dir = None
    samples_dir = None
    sample_every = 1
    num_workers = 0  # for DataLoader

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

In [22]:
class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config, args):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config
        self.iters = 0
        self.fixed_x = None
        self.fixed_y = None
        print("Using wandb")
        wandb.init(project='LayoutTransformer-finetune0511', name=args["exp"])
        wandb.config.update(args)

        # take over whatever gpus are on the system
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)

    def save_checkpoint(self):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        ckpt_path = os.path.join(self.config.ckpt_dir, 'checkpoint.pth')
        logger.info("saving %s", ckpt_path)
        torch.save(raw_model.state_dict(), ckpt_path)
        wandb.save(ckpt_path)

    def train(self):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)
        pad_token = self.train_dataset.vocab_size - 1

        def run_epoch(split):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, shuffle=True, pin_memory=True,
                                batch_size=config.batch_size,
                                num_workers=config.num_workers)

            losses = []
            #pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            pbar = tqdm(enumerate(loader), total=len(loader))
            for it, (x, y) in pbar:

                if epoch == 0 and not is_train:
                    self.fixed_x = x[:min(10, len(x))]
                    self.fixed_y = y[:min(10, len(y))]
                    #self.fixed_x = x
                    #self.fixed_y = y
                    print("self.fixed_x is ", self.fixed_x.shape)
                    print("self.fixed_y is ", self.fixed_y.shape)

                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    # import ipdb; ipdb.set_trace()
                    logits, loss = model(x, y, pad_token=pad_token)
                    loss = loss.mean()  # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())
                    del x, y

                if is_train:

                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()
                    self.iters += 1
                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        # self.tokens += (y >= 0).sum()  # number of tokens processed this step (i.e. label is not -100)
                        if self.iters < config.warmup_iters:
                            # linear warmup
                            lr_mult = float(self.iters) / float(max(1, config.warmup_iters))
                        else:
                            # cosine learning rate decay
                            progress = float(self.iters - config.warmup_iters) / float(max(1, config.final_iters - config.warmup_iters))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    wandb.log({
                        'train loss': loss.item(),
                        'lr': lr, 'epoch': epoch+1
                    }, step=self.iters)
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
                else:
                    pbar.set_description(f"epoch {epoch+1} iter {it}: test loss {loss.item():.5f}")
                del loss
            if not is_train:
                test_loss = float(np.mean(losses))
                logger.info("test loss: %f", test_loss)
                wandb.log({'test loss': test_loss}, step=self.iters)
                #pbar.set_description(f"epoch {epoch+1} iter {it}: test loss {test_loss.item():.5f}")
                return test_loss

        best_loss = float('inf')
        for epoch in range(config.max_epochs):
            run_epoch('train')
            if self.test_dataset is not None:
                with torch.no_grad():
                    test_loss = run_epoch('test')

            # supports early stopping based on the test loss, or just save always if no test set is provided
            good_model = self.test_dataset is None or test_loss < best_loss
            if self.config.ckpt_dir is not None and good_model:
                best_loss = test_loss
                self.save_checkpoint()

            # sample from the model
            if self.config.samples_dir is not None and (epoch+1) % self.config.sample_every == 0:
                # import ipdb; ipdb.set_trace()
                # inputs
                #print("self.fixed_x is ", self.fixed_x)
                layouts = self.fixed_x.detach().cpu().numpy()
                input_layouts = [self.train_dataset.render(layout)[0] for layout in layouts]
                overlap_rates = np.array([self.train_dataset.render(layout)[1] for layout in layouts])
                coverages = np.array([self.train_dataset.render(layout)[2] for layout in layouts])
                print("overlap_rates ", overlap_rates)
                print("coverages ", coverages)
                print("statsitic average of overlap_rates is ", np.average(overlap_rates))
                print("statsitic std of overlap_rates is ", np.std(overlap_rates))
                print("statsitic average of coverages is ", np.average(coverages))
                print("statsitic std of coverages is ", np.std(coverages))
                #print("input layouts is ", layouts)
                #print("input layouts is ", input_layouts)
                # for i, layout in enumerate(layouts):
                #     layout = self.train_dataset.render(layout)
                #     layout.save(os.path.join(self.config.samples_dir, f'input_{epoch:02d}_{i:02d}.png'))

                # reconstruction
                x_cond = self.fixed_x.to(self.device)
                logits, _ = model(x_cond)
                probs = F.softmax(logits, dim=-1)
                _, y = torch.topk(probs, k=1, dim=-1)
                layouts = torch.cat((x_cond[:, :1], y[:, :, 0]), dim=1).detach().cpu().numpy()
                recon_layouts = [self.train_dataset.render(layout)[0] for layout in layouts]
                recon_overlap_rates = [self.train_dataset.render(layout)[1] for layout in layouts]
                print("recon_overlap_rates ", recon_overlap_rates)
                recon_coverages = [self.train_dataset.render(layout)[2] for layout in layouts]
                print("recon_coverages ", recon_coverages)
                print("statsitic average of recon_overlap_rates is ", np.average(recon_overlap_rates))
                print("statsitic std of  recon_overlap_rates is ", np.std( recon_overlap_rates))
                print("statsitic average of recon_coverages is ", np.average(recon_coverages))
                print("statsitic std of recon_coverages is ", np.std(recon_coverages))
                # for i, layout in enumerate(layouts):
                #     layout = self.train_dataset.render(layout)
                #     layout.save(os.path.join(self.config.samples_dir, f'recon_{epoch:02d}_{i:02d}.png'))

                # samples - random
                layouts = sample(model, x_cond[:, :6], steps=self.train_dataset.max_length,
                                 temperature=1.0, sample=True, top_k=5).detach().cpu().numpy()
                sample_random_layouts = [self.train_dataset.render(layout)[0] for layout in layouts]
                sample_random_overlap_rates = np.array([self.train_dataset.render(layout)[1] for layout in layouts])
                sample_random_coverages = np.array([self.train_dataset.render(layout)[2] for layout in layouts])
                print("sample_random overlap_rates ", sample_random_overlap_rates)
                print("sample_random coverages ", sample_random_coverages)
                print("statsitic average of sample_random overlap_rates is ", np.average(sample_random_overlap_rates))
                print("statsitic std of sample_random overlap_rates is ", np.std(sample_random_overlap_rates))
                print("statsitic average of sample_random coverages is ", np.average(sample_random_coverages))
                print("statsitic std of sample_random coverages is ", np.std(sample_random_coverages))
                # for i, layout in enumerate(layouts):
                #     layout = self.train_dataset.render(layout)
                #     layout.save(os.path.join(self.config.samples_dir, f'sample_random_{epoch:02d}_{i:02d}.png'))

                # samples - deterministic
                layouts = sample(model, x_cond[:, :6], steps=self.train_dataset.max_length,
                                 temperature=1.0, sample=False, top_k=None).detach().cpu().numpy()
                sample_det_layouts = [self.train_dataset.render(layout)[0] for layout in layouts]
                sample_det_overlap_rates = np.array([self.train_dataset.render(layout)[1] for layout in layouts])
                sample_det_coverages = np.array([self.train_dataset.render(layout)[2] for layout in layouts])
                print("sample_det overlap_rates ", sample_det_overlap_rates)
                print("sample_det coverages ", sample_det_coverages)
                print("statsitic average of sample_det overlap_rates is ", np.average(sample_det_overlap_rates))
                print("statsitic std of sample_det overlap_rates is ", np.std(sample_det_overlap_rates))
                print("statsitic average of sample_det coverages is ", np.average(sample_det_coverages))
                print("statsitic std of sample_det coverages is ", np.std(sample_det_coverages))
                # for i, layout in enumerate(layouts):
                #     layout = self.train_dataset.render(layout)
                #     layout.save(os.path.join(self.config.samples_dir, f'sample_det_{epoch:02d}_{i:02d}.png'))

                wandb.log({
                    "input_layouts": [wandb.Image(pil, caption=f'input_{epoch:02d}_{i:02d}.png')
                                      for i, pil in enumerate(input_layouts)],
                    "recon_layouts": [wandb.Image(pil, caption=f'recon_{epoch:02d}_{i:02d}.png')
                                      for i, pil in enumerate(recon_layouts)],
                    "overlap_rates": overlap_rates,
                    "coverages": coverages,
                    "recon_overlap_rates": recon_overlap_rates,
                    "recon_coverages": recon_coverages,
                    "sample_random_layouts": [wandb.Image(pil, caption=f'sample_random_{epoch:02d}_{i:02d}.png')
                                              for i, pil in enumerate(sample_random_layouts)],
                    "sample_det_layouts": [wandb.Image(pil, caption=f'sample_det_{epoch:02d}_{i:02d}.png')
                                          for i, pil in enumerate(sample_det_layouts)],
                }, step=self.iters)

In [23]:
log_dir = os.path.join(args["log_dir"], args["exp"])
samples_dir = os.path.join(log_dir, "samples")
ckpt_dir = os.path.join(log_dir, "checkpoints")
os.makedirs(samples_dir, exist_ok=True)
os.makedirs(ckpt_dir, exist_ok=True)

In [24]:
set_seed(args["seed"])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device: cuda


In [25]:
tconf = TrainerConfig(max_epochs=args["epochs"],
                          batch_size=args["batch_size"],
                          lr_decay=args["lr_decay"],
                          learning_rate=args["lr"] * args["batch_size"],
                          warmup_iters=args["warmup_iters"],
                          final_iters=args["final_iters"],
                          ckpt_dir=ckpt_dir,
                          samples_dir=samples_dir,
                          sample_every=args["sample_every"])
trainer = Trainer(model, train_dataset, valid_dataset, tconf, args)

Using wandb


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
trainer.train()

epoch 1 iter 14: train loss 3.40439. lr 7.200000e-05: 100%|██████████| 15/15 [00:03<00:00,  3.78it/s]
epoch 1 iter 1: test loss 3.39540:  50%|█████     | 2/4 [00:00<00:00, 11.75it/s]

self.fixed_x is  torch.Size([10, 517])
self.fixed_y is  torch.Size([10, 517])
self.fixed_x is  torch.Size([10, 517])
self.fixed_y is  torch.Size([10, 517])
self.fixed_x is  torch.Size([10, 517])
self.fixed_y is  torch.Size([10, 517])


epoch 1 iter 3: test loss 3.27784: 100%|██████████| 4/4 [00:00<00:00, 12.41it/s]


self.fixed_x is  torch.Size([10, 517])
self.fixed_y is  torch.Size([10, 517])


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


...

overlap_rates  [0.47508865 0.         0.39198964 0.05835423 0.01272178 0.
 0.         0.343079   0.4229443  0.        ]
coverages  [0.8605957  0.75019836 0.74252319 0.80328369 0.54093933 0.56816101
 0.69799805 0.64454651 0.80535889 0.6204834 ]
statsitic average of overlap_rates is  0.1704177604884798
statsitic std of overlap_rates is  0.19725494030364016
statsitic average of coverages is  0.7034088134765625
statsitic std of coverages is  0.10190912439987176
recon_overlap_rates  [1, 0.4406347819756861, 0.4623848404139196, 0.47630324650019484, 1, 1, 0.11771447834147986, 1, 0.2612745098039216, 0.6994497248624312]
recon_coverages  [0.2320404052734375, 0.419219970703125, 0.3494720458984375, 0.5090179443359375, 0.2936248779296875, 0.4431610107421875, 0.4526519775390625, 0.5252685546875, 0.37353515625, 0.4575347900390625]
statsitic average of recon_overlap_rates is  0.6457761581897634
statsitic std of  recon_overlap_rates is  0.32185494481043886
statsitic average of recon_coverages is  0.4055

epoch 99 iter 14: train loss 0.09302. lr 7.200000e-05: 100%|██████████| 15/15 [00:03<00:00,  3.84it/s]
epoch 99 iter 3: test loss 5.95830: 100%|██████████| 4/4 [00:00<00:00, 12.63it/s]


overlap_rates  [0.47508865 0.         0.39198964 0.05835423 0.01272178 0.
 0.         0.343079   0.4229443  0.        ]
coverages  [0.8605957  0.75019836 0.74252319 0.80328369 0.54093933 0.56816101
 0.69799805 0.64454651 0.80535889 0.6204834 ]
statsitic average of overlap_rates is  0.1704177604884798
statsitic std of overlap_rates is  0.19725494030364016
statsitic average of coverages is  0.7034088134765625
statsitic std of coverages is  0.10190912439987176
recon_overlap_rates  [0.5549540426857766, 0.5050963025446366, 0.7622422680412371, 0.5110857417532897, 1, 1, 0.32293903145554304, 1, 0.21408150040025922, 0.5752945839789557]
recon_coverages  [0.39178466796875, 0.43414306640625, 0.2841796875, 0.507904052734375, 0.4071044921875, 0.4818878173828125, 0.4477386474609375, 0.5086669921875, 0.4002838134765625, 0.4234466552734375]
statsitic average of recon_overlap_rates is  0.6445693470859697
statsitic std of  recon_overlap_rates is  0.27050066693595515
statsitic average of recon_coverages i

epoch 100 iter 14: train loss 0.09419. lr 7.200000e-05: 100%|██████████| 15/15 [00:03<00:00,  3.87it/s]
epoch 100 iter 3: test loss 5.85859: 100%|██████████| 4/4 [00:00<00:00, 12.62it/s]


overlap_rates  [0.47508865 0.         0.39198964 0.05835423 0.01272178 0.
 0.         0.343079   0.4229443  0.        ]
coverages  [0.8605957  0.75019836 0.74252319 0.80328369 0.54093933 0.56816101
 0.69799805 0.64454651 0.80535889 0.6204834 ]
statsitic average of overlap_rates is  0.1704177604884798
statsitic std of overlap_rates is  0.19725494030364016
statsitic average of coverages is  0.7034088134765625
statsitic std of coverages is  0.10190912439987176
recon_overlap_rates  [0.9429859578265157, 0.8155568650559811, 0.37341271932135517, 0.6763805678562126, 0.9212538979156409, 1, 0.4967384086559119, 1, 0.34961167333490234, 0.4973018600584848]
recon_coverages  [0.3249053955078125, 0.414306640625, 0.2895965576171875, 0.4686279296875, 0.4648590087890625, 0.336029052734375, 0.3906402587890625, 0.4202423095703125, 0.389007568359375, 0.5061492919921875]
statsitic average of recon_overlap_rates is  0.7073241950025004
statsitic std of  recon_overlap_rates is  0.24762942740544167
statsitic ave

In [ ]:
trainer.train()

epoch 1 iter 7: train loss 3.33517. lr 1.440000e-04: 100%|██████████| 8/8 [00:02<00:00,  3.61it/s]
epoch 1 iter 1: test loss 3.31453: 100%|██████████| 2/2 [00:00<00:00, 11.50it/s]
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


self.fixed_x is  tensor([[261, 257,  33,  ..., 263, 263, 263],
        [261, 257,  10,  ..., 263, 263, 263],
        [261, 257, 203,  ..., 263, 263, 263],
        [261, 257,  11,  ..., 263, 263, 263]])
input layouts is  [[261 257  33 ... 263 263 263]
 [261 257  10 ... 263 263 263]
 [261 257 203 ... 263 263 263]
 [261 257  11 ... 263 263 263]]
input layouts is  [<PIL.Image.Image image mode=RGB size=260x260 at 0x7F68F0DAC0D0>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E80686D0>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E8068950>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E80681D0>]


epoch 2 iter 7: train loss 3.20320. lr 1.440000e-04: 100%|██████████| 8/8 [00:02<00:00,  3.92it/s]
epoch 2 iter 1: test loss 3.17819: 100%|██████████| 2/2 [00:00<00:00, 11.49it/s]
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


self.fixed_x is  tensor([[261, 257,  33,  ..., 263, 263, 263],
        [261, 257,  10,  ..., 263, 263, 263],
        [261, 257, 203,  ..., 263, 263, 263],
        [261, 257,  11,  ..., 263, 263, 263]])
input layouts is  [[261 257  33 ... 263 263 263]
 [261 257  10 ... 263 263 263]
 [261 257 203 ... 263 263 263]
 [261 257  11 ... 263 263 263]]
input layouts is  [<PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0683310>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E06831D0>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0683050>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E075DE50>]


epoch 3 iter 7: train loss 3.03667. lr 1.440000e-04: 100%|██████████| 8/8 [00:02<00:00,  3.94it/s]
epoch 3 iter 1: test loss 3.19365: 100%|██████████| 2/2 [00:00<00:00, 11.50it/s]
wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


...

self.fixed_x is  tensor([[261, 257,  33,  ..., 263, 263, 263],
        [261, 257,  10,  ..., 263, 263, 263],
        [261, 257, 203,  ..., 263, 263, 263],
        [261, 257,  11,  ..., 263, 263, 263]])
input layouts is  [[261 257  33 ... 263 263 263]
 [261 257  10 ... 263 263 263]
 [261 257 203 ... 263 263 263]
 [261 257  11 ... 263 263 263]]
input layouts is  [<PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E015CE10>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E015CA10>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E05AFC90>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E9550BD0>]


epoch 99 iter 7: train loss 0.07332. lr 1.440000e-04: 100%|██████████| 8/8 [00:02<00:00,  3.93it/s]
epoch 99 iter 1: test loss 6.05944: 100%|██████████| 2/2 [00:00<00:00, 11.52it/s]


self.fixed_x is  tensor([[261, 257,  33,  ..., 263, 263, 263],
        [261, 257,  10,  ..., 263, 263, 263],
        [261, 257, 203,  ..., 263, 263, 263],
        [261, 257,  11,  ..., 263, 263, 263]])
input layouts is  [[261 257  33 ... 263 263 263]
 [261 257  10 ... 263 263 263]
 [261 257 203 ... 263 263 263]
 [261 257  11 ... 263 263 263]]
input layouts is  [<PIL.Image.Image image mode=RGB size=260x260 at 0x7F68F0C94710>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0358550>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0489B90>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0489C50>]


epoch 100 iter 7: train loss 0.08281. lr 1.440000e-04: 100%|██████████| 8/8 [00:02<00:00,  3.93it/s]
epoch 100 iter 1: test loss 6.10852: 100%|██████████| 2/2 [00:00<00:00, 11.48it/s]


self.fixed_x is  tensor([[261, 257,  33,  ..., 263, 263, 263],
        [261, 257,  10,  ..., 263, 263, 263],
        [261, 257, 203,  ..., 263, 263, 263],
        [261, 257,  11,  ..., 263, 263, 263]])
input layouts is  [[261 257  33 ... 263 263 263]
 [261 257  10 ... 263 263 263]
 [261 257 203 ... 263 263 263]
 [261 257  11 ... 263 263 263]]
input layouts is  [<PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0489650>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0248950>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E0248890>, <PIL.Image.Image image mode=RGB size=260x260 at 0x7F68E02488D0>]


In [ ]:
publeynet_train_dataset = JSONLayout("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Publeynet/train.json")

images is  <class 'list'>
annotations is  <class 'list'>
categories is  <class 'list'>


In [ ]:
publeynet_train_dataset.size

256

In [ ]:
publeynet_train_dataset.vocab_size

264

In [ ]:
publeynet_train_dataset.max_length

517

In [ ]:
pubf = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Publeynet/val.json")
pubdata = json.load(pubf)

In [ ]:
pubdata['categories']

[{'id': 1, 'name': 'text', 'supercategory': ''},
 {'id': 2, 'name': 'title', 'supercategory': ''},
 {'id': 3, 'name': 'list', 'supercategory': ''},
 {'id': 4, 'name': 'table', 'supercategory': ''},
 {'id': 5, 'name': 'figure', 'supercategory': ''}]

In [ ]:
f = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/poster_dataset.json")
data = json.load(f)
convert_dict = {1: 2, 2: 1, 3: 5, 4: 1, 5: 1, 6: 5, 7: 4}
annotations = []
print("previously, data['categories'] is ", type(data['categories']))
print("previously, data['categories'] is ", data['categories'])
data['categories'] = [{'id': 1, 'name': 'text', 'supercategory': ''},
                      {'id': 2, 'name': 'title', 'supercategory': ''},
                      {'id': 3, 'name': 'list', 'supercategory': ''},
                      {'id': 4, 'name': 'table', 'supercategory': ''},
                      {'id': 5, 'name': 'figure', 'supercategory': ''}]
print("After, data['categories'] is ", type(data['categories']))
print("After, data['categories'] is ", data['categories'])
exit(0)
for item in data['annotations']:
    item['category_id'] = convert_dict[item['category_id']]

valid_images = random.sample(data["images"], 60)
valid_images_id = set([item["id"] for item in valid_images])
train_images = [item for item in data["images"] if item["id"] not in valid_images_id]
train_images_id = set([item["id"] for item in train_images])
train_annotaitons = [item for item in data["annotations"] if item['image_id'] in train_images_id]
valid_annotaitons = [item for item in data["annotations"] if item['image_id'] in valid_images_id]
#train_data = {'images': train_images, 'annotations': train_annotaitons, 'categories': data['categories']}
train_data = {'images': train_images}
train_data = {"images": train_images, "annotations": train_annotaitons, "categories": data['categories']}
#valid_data = {'images': valid_images, 'annotations': valid_annotaitons, 'categories': data['categories']}
trainf = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/poster_finetune_train.json", 'w')
json.dump(train_data, trainf)
trainf.close()
#validf = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/poster_finetune_valid.json", 'w')
#json.dump(valid_data, validf)
validf.close()

previously, data['categories'] is  <class 'list'>
previously, data['categories'] is  [{'supercategory': '', 'id': 1, 'name': 'title'}, {'supercategory': '', 'id': 2, 'name': 'author'}, {'supercategory': '', 'id': 3, 'name': 'logo'}, {'supercategory': '', 'id': 4, 'name': 'section_header'}, {'supercategory': '', 'id': 5, 'name': 'section_body'}, {'supercategory': '', 'id': 6, 'name': 'figure'}, {'supercategory': '', 'id': 7, 'name': 'table'}]
After, data['categories'] is  <class 'list'>
After, data['categories'] is  [{'supercategory': '', 'id': 1, 'name': 'text'}, {'supercategory': '', 'id': 2, 'name': 'title'}, {'supercategory': '', 'id': 3, 'name': 'list'}, {'supercategory': '', 'id': 4, 'name': 'table'}, {'supercategory': '', 'id': 5, 'name': 'figure'}]


In [ ]:
f = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/poster_finetune_train.json")
strings = f.read()
json.loads(strings)

JSONDecodeError: ignored

In [ ]:
train_data['categories']

[{'id': 1, 'name': 'text', 'supercategory': ''},
 {'id': 2, 'name': 'title', 'supercategory': ''},
 {'id': 3, 'name': 'list', 'supercategory': ''},
 {'id': 4, 'name': 'table', 'supercategory': ''},
 {'id': 5, 'name': 'figure', 'supercategory': ''}]

In [ ]:
valid_data = {"images": valid_images, "annotations": valid_annotaitons,

In [ ]:
valid_images = random.sample(data["images"], 60)
valid_images_id = set([item["id"] for item in valid_images])
train_images = [item for item in data["images"] if item["id"] not in valid_images_id]
train_images_id = set([item["id"] for item in train_images])

In [ ]:
train_annotaitons = [item for item in data["annotations"] if item['image_id'] in train_images_id]
valid_annotaitons = [item for item in data["annotations"] if item['image_id'] in valid_images_id]

In [ ]:
train_data = {"images": train_images, "annotations": train_annotaitons, "categories": data['categories']}
valid_data = {"images": valid_images, "annotations": valid_annotaitons, "categories": data['categories']}

In [ ]:
trainf = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/poster_train.json", 'w')
json.dump(train_data, trainf)

In [ ]:
validf = open("/content/drive/MyDrive/Directed_Study/DeepLayout-main/Poster-Data/poster_valid.json", 'w')
json.dump(valid_data, validf)

In [ ]:
len(valid_annotaitons)

1528